# Normal Traning

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

In [1]:
import sys
sys.path.append('../modules')
from custom_dropout import DeterministicDropout
from model_wrapper import NetWrapper
from import_data import load_cifar
from misc import write_to_json
from torch import nn, optim
import torch
import pandas as pd
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import os
import json
from torchvision.models import efficientnet_b4, efficientnet_b0
from torch import nn


/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b4, efficientnet_b0

class TeacherNet(nn.Module):
    def __init__(self, dropout):
        super(TeacherNet, self).__init__()
        self.dropout = dropout

        # Load the EfficientNet-B4 model
        self.backbone = efficientnet_b4(weights='IMAGENET1K_V1')

        # Modify the classifier
        self.backbone.classifier = nn.Sequential(
            nn.Linear(self.backbone.classifier[1].in_features, 512),
            nn.ReLU(),
            self.dropout,  # Add the dropout layer
            nn.Linear(512, 10)  # Output layer for 10 classes
        )

    def forward(self, input_data):
        return self.backbone(input_data)


class StudentNet(nn.Module):
    def __init__(self, dropout):
        super(StudentNet, self).__init__()
        self.dropout = dropout

        # Load the EfficientNet-B0 model
        self.backbone = efficientnet_b0(weights=None)

        # Modify the classifier
        self.backbone.classifier = nn.Sequential(
            nn.Linear(self.backbone.classifier[1].in_features, 256),
            nn.ReLU(),
            self.dropout,  # Add the dropout layer
            nn.Linear(256, 10)  # Output layer for 10 classes
        )

    def forward(self, input_data):
        return self.backbone(input_data)


In [5]:
batch_size = 128
epoch = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
teacher_model = TeacherNet(nn.Dropout(0.5)).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
teacher_wrapper.fit(trainloader, validationloader, num_epochs=50, verbose=True, patience=5)

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 1.446757922795686
Training Accuracy in epoch 1 :::: 49.73
Validation loss in epoch 1 :::: 1.4452325642108916
Validation Accuracy in epoch 1 :::: 49.46
Time Elapsed: 72.34s
Training loss in epoch 2 :::: 1.1476493135771968
Training Accuracy in epoch 2 :::: 59.96
Validation loss in epoch 2 :::: 1.231582573056221
Validation Accuracy in epoch 2 :::: 56.22
Time Elapsed: 71.33s
Training loss in epoch 3 :::: 0.9405790462412618
Training Accuracy in epoch 3 :::: 67.82
Validation loss in epoch 3 :::: 1.035557173192501
Validation Accuracy in epoch 3 :::: 62.74
Time Elapsed: 72.45s
Training loss in epoch 4 :::: 0.761100632223216
Training Accuracy in epoch 4 :::: 73.14
Validation loss in epoch 4 :::: 0.9317599691450595
Validation Accuracy in epoch 4 :::: 66.46
Time Elapsed: 70.67s
Training loss in epoch 5 :::: 0.6672021165828813
Training Accuracy in epoch 5 :::: 76.74
Validation loss in epoch 5 

In [ ]:
torch.save(teacher_model.state_dict(), "../output/EfficentNetB4-EfficentNetB4-CIFAR10/normal-training/EfficentNetB4-EfficentNetB0-cifar10-teacher_best.pth")

In [10]:
teacher_dropout = nn.Dropout(p=0.5)
teacher_model = TeacherNet(teacher_dropout).to(device)
teacher_model.load_state_dict(torch.load(os.path.join("../output/EfficentNetB4-EfficentNetB4-CIFAR10/normal-training/EfficentNetB4-EfficentNetB0-cifar10-teacher_best.pth")))
teacher_model.eval()

# Đánh giá trên test set
accuracy, loss, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)

# In các chỉ số đánh giá
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Loss: {loss:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Per-class Accuracy:")
for i, acc in enumerate(per_class_acc):
    print(f"  Class {i}: {acc:.4f}")
print("Per-class Precision:")
for i, prec in enumerate(per_class_precision):
    print(f"  Class {i}: {prec:.4f}")


Test Accuracy: 0.8035
Test Loss: 0.9276
Confusion Matrix:
[[844.  13.  42.  12.  19.   1.   5.   8.  37.  19.]
 [  9. 906.   1.   2.   1.   1.   2.   3.  32.  43.]
 [ 27.   2. 785.  43.  52.  24.  40.  21.   2.   4.]
 [ 11.   5.  70. 678.  35. 126.  40.  19.   9.   7.]
 [  8.   2.  53.  32. 818.  22.  40.  20.   3.   2.]
 [ 10.   3.  39. 190.  32. 666.  18.  32.   7.   3.]
 [  7.  11.  35.  29.  21.  19. 866.   2.   8.   2.]
 [ 12.   2.  16.  42.  46.  25.   3. 843.   3.   8.]
 [ 66.  21.  15.   7.   9.   4.   1.   3. 867.   7.]
 [ 38.  69.   4.   6.   3.   3.   2.   8. 105. 762.]]
Per-class Accuracy:
  Class 0: 0.8440
  Class 1: 0.9060
  Class 2: 0.7850
  Class 3: 0.6780
  Class 4: 0.8180
  Class 5: 0.6660
  Class 6: 0.8660
  Class 7: 0.8430
  Class 8: 0.8670
  Class 9: 0.7620
Per-class Precision:
  Class 0: 0.8178
  Class 1: 0.8762
  Class 2: 0.7406
  Class 3: 0.6513
  Class 4: 0.7896
  Class 5: 0.7475
  Class 6: 0.8515
  Class 7: 0.8790
  Class 8: 0.8080
  Class 9: 0.8891


In [11]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
output_path = f'../output/EfficentNetB4-EfficentNetB4-CIFAR10/normal-training/EfficentNetB4-EfficentNetB4-CIFAR10-teacher_best.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

### Normal KD Zone

In [12]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [16]:
batch_size = 128
epochs = 50
alphas = [0.3, 0.5, 0.7]
temperature = 20
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


for alpha in alphas:
    teacher_dropout = nn.Dropout(p=0.5)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load(os.path.join("../output/EfficentNetB4-EfficentNetB4-CIFAR10/normal-training/EfficentNetB4-EfficentNetB0-cifar10-teacher_best.pth")))
    teacher_model.eval()
    
    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    
    # Train Student
    student_model.train()
    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 5  # stop if val_acc doesn’t improve for 3 epochs
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()
            
            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)
            
            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()
            running_loss += loss.item()
            
        
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())
    
        # Calculate metrics
        train_loss_avg = running_loss / len(trainloader)
        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")
        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            #torch.save(student_model.state_dict(), os.path.join(log_dir, "alpha_0.3_dropout_0.3.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break
        # Validation loop for metrics
    
        student_model.train()
    
    output_path = f'../output/EfficentNetB4-EfficentNetB4-CIFAR10/normal-training/KD_normal_alpha_{alpha}.json'
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)
    write_to_json(
                output_path,
                'student',
                student_wrapper,
                accuracy,
                conf_matrix,
                per_class_acc,
                per_class_precision,
                classes)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/50] - Loss: 22.8238, Validation Accuracy: 0.3806, Validation F1 Score: 0.3584
Epoch [2/50] - Loss: 18.7233, Validation Accuracy: 0.4468, Validation F1 Score: 0.4459
Epoch [3/50] - Loss: 16.3142, Validation Accuracy: 0.5372, Validation F1 Score: 0.5335
Epoch [4/50] - Loss: 14.5820, Validation Accuracy: 0.5586, Validation F1 Score: 0.5589
Epoch [5/50] - Loss: 13.3759, Validation Accuracy: 0.5832, Validation F1 Score: 0.5788
Epoch [6/50] - Loss: 12.3536, Validation Accuracy: 0.6148, Validation F1 Score: 0.6165
Epoch [7/50] - Loss: 11.6174, Validation Accuracy: 0.6322, Validation F1 Score: 0.6352
Epoch [8/50] - Loss: 11.1306, Validation Accuracy: 0.6264, Validation F1 Score: 0.6214
No improvement for 1 epoch(s).
Epoch [9/50] - Loss: 11.0301, Validation Accuracy: 0.6500, Validation F1 Score: 0.6527
Epoch [10/50] - Loss: 10.0778, Validation Accuracy: 0.6532, Validation F1 Score: 0.6597
Epoch [11/50] - Loss: 

# A. Min-Activation

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Chọn GPU đầu tiên

In [10]:
import sys
sys.path.append('../modules')
from custom_dropout import DeterministicDropout
#from new_distilation_model import TeacherNet, StudentNet
from model_wrapper import NetWrapper
from import_data import load_cifar
from misc import write_to_json
from torch import nn, optim
import torch
import pandas as pd
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import os
import json
from torchvision.models import resnet50, resnet18
from torch import nn

In [11]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [12]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b4, efficientnet_b0

class TeacherNet(nn.Module):
    def __init__(self, dropout):
        super(TeacherNet, self).__init__()
        self.dropout = dropout

        # Load the EfficientNet-B4 model
        self.backbone = efficientnet_b4(weights='IMAGENET1K_V1')

        # Modify the classifier
        self.backbone.classifier = nn.Sequential(
            nn.Linear(self.backbone.classifier[1].in_features, 512),
            nn.ReLU(),
            self.dropout,  # Add the dropout layer
            nn.Linear(512, 10)  # Output layer for 10 classes
        )

    def forward(self, input_data):
        return self.backbone(input_data)


class StudentNet(nn.Module):
    def __init__(self, dropout):
        super(StudentNet, self).__init__()
        self.dropout = dropout

        # Load the EfficientNet-B0 model
        self.backbone = efficientnet_b0(weights=None)

        # Modify the classifier
        self.backbone.classifier = nn.Sequential(
            nn.Linear(self.backbone.classifier[1].in_features, 256),
            nn.ReLU(),
            self.dropout,  # Add the dropout layer
            nn.Linear(256, 10)  # Output layer for 10 classes
        )

    def forward(self, input_data):
        return self.backbone(input_data)


In [ ]:
batch_size = 128
epochs = 50
dropout_rate = [0.1, 0.3 , 0.5]
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
for r in dropout_rate:
    print(f"-------------- Droptout rate: {r} --------------")
    teacher_dropout = DeterministicDropout('max_activation', r)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    teacher_wrapper.fit(trainloader, validationloader, num_epochs=epochs, verbose=True, patience=5)
    torch.save(teacher_model.state_dict(),f"../output/EfficentNetB4-EfficentNetB4-CIFAR10/min-activation/MA-teacher_model-dropoutRate_{r}.pth")
    output_path = f'../output/EfficentNetB4-EfficentNetB4-CIFAR10/min-activation/MA-teacher_model-dropoutRate_{r}.json'
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)
    write_to_json(
        output_path,
        'model',
        teacher_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

### A.KD Zone

In [4]:
import torch.nn.functional as F
 
ce_loss = nn.CrossEntropyLoss()
 
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    # Softmax + Log-Softmax
    soft_targets = F.softmax(teacher_logits / temperature, dim=-1)
    soft_log_probs = F.log_softmax(student_logits / temperature, dim=-1)
 
    # KL divergence theo paper: H(p, q) = Σ p(x) * [log(p(x)) - log(q(x))]
    soft_loss = torch.sum(soft_targets * (torch.log(soft_targets + 1e-8) - soft_log_probs), dim=1).mean()
    soft_loss *= temperature ** 2
 
    # Cross entropy loss
    label_loss = ce_loss(student_logits, labels)
 
    # Weighted sum
    loss = alpha * soft_loss + (1 - alpha) * label_loss
    return loss

In [5]:
batch_size = 128
epochs =50
alphas = [0.3, 0.5, 0.7]
temperature = 20
dropout_rate = [0.1, 0.3, 0.5]
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
for alpha in alphas:
    for r in dropout_rate:
        print(f"-------------- Alpha: {alpha} - Droptout rate: {r} --------------")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        teacher_dropout = DeterministicDropout('max_activation', r)
        teacher_model = TeacherNet(teacher_dropout).to(device)
        teacher_model.load_state_dict(torch.load(os.path.join(f"../output/EfficentNetB4-EfficentNetB4-CIFAR10/min-activation/MA-teacher_model-dropoutRate_{r}.pth")))
        teacher_model.eval()
        
        # Student Model
        student_dropout = nn.Dropout(p=0.5)
        student_model = StudentNet(student_dropout).to(device)
        student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
        
        # Train Student
        student_model.train()
        best_val_accuracy = 0
        patience = 5  
        counter = 0 
        
        for epoch in range(epochs):
            running_loss = 0.0
            for inputs, labels in trainloader:
                inputs, labels = inputs.to(device), labels.to(device)
                #print (labels)
                student_optimizer.zero_grad()
                with torch.no_grad():
                    teacher_outputs = teacher_model(inputs)
                    #print(teacher_outputs)
                student_outputs = student_model(inputs)
                #print(student_outputs)
                loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                #print(loss)
                loss.backward()
                student_optimizer.step()
                running_loss += loss.item()
        
            # Validation loop for metrics
            student_model.eval()
            val_preds = []
            val_labels = []
            with torch.no_grad():
                for val_inputs, val_targets in validationloader:
                    val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                    val_outputs = student_model(val_inputs)
                    _, preds = torch.max(val_outputs, 1)
                    val_preds.extend(preds.cpu().numpy())
                    val_labels.extend(val_targets.cpu().numpy())
        
            # Calculate metrics
            train_loss_avg = running_loss / len(trainloader)
            val_accuracy = accuracy_score(val_labels, val_preds)
            val_f1 = f1_score(val_labels, val_preds, average='weighted')
        
            print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
                  f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")
        
            # --- EARLY STOPPING ---
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                counter = 0  # Reset counter nếu có cải thiện
            else:
                counter += 1
                print(f"EarlyStopping counter: {counter} out of {patience}")
                if counter >= patience:
                    print("Early stopping triggered.")
                    break
            # ----------------------
        
            student_model.train()
        
        #Save final Student metrics
        output_path =f'../output/EfficentNetB4-EfficentNetB4-CIFAR10/min-activation/alpha{alpha}_dropout{r}.json'
        
        student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
        accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)
        write_to_json(
                    output_path,
                    'student',
                    student_wrapper,
                    accuracy,
                    conf_matrix,
                    per_class_acc,
                    per_class_precision,
                    classes
                )

Files already downloaded and verified
Files already downloaded and verified
-------------- Alpha: 0.3 - Droptout rate: 0.1 --------------
Epoch [1/50] - Loss: 39.1361, Validation Accuracy: 0.3666, Validation F1 Score: 0.3270
Epoch [2/50] - Loss: 32.9213, Validation Accuracy: 0.4610, Validation F1 Score: 0.4593
Epoch [3/50] - Loss: 28.8952, Validation Accuracy: 0.4926, Validation F1 Score: 0.4867
Epoch [4/50] - Loss: 26.5097, Validation Accuracy: 0.5362, Validation F1 Score: 0.5427
Epoch [5/50] - Loss: 24.1527, Validation Accuracy: 0.5704, Validation F1 Score: 0.5733
Epoch [6/50] - Loss: 22.6257, Validation Accuracy: 0.5544, Validation F1 Score: 0.5563
EarlyStopping counter: 1 out of 3
Epoch [7/50] - Loss: 21.2563, Validation Accuracy: 0.5786, Validation F1 Score: 0.5789
Epoch [8/50] - Loss: 20.2836, Validation Accuracy: 0.5842, Validation F1 Score: 0.5905
Epoch [9/50] - Loss: 19.0494, Validation Accuracy: 0.5924, Validation F1 Score: 0.6008
Epoch [10/50] - Loss: 18.0410, Validation Acc

# B. Sample-Droping

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('../modules')
from torch import nn, optim
from torchvision.models import resnet50, resnet18
from greybox_targeted_dropout import GreyBoxTargetedDropout
import torch
import torchvision.transforms as transforms
from model_wrapper_gt import NetWrapper_T
from import_data import load_cifar
from misc import write_to_json
import os
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b4, efficientnet_b0

class TeacherNet_SD(nn.Module):
    def __init__(self, dropout_layer):
        super(TeacherNet_SD, self).__init__()
        self.effnet = efficientnet_b4(weights='IMAGENET1K_V1')
        self.dropout_layer = dropout_layer

        # Modify the fully connected (classifier) layer
        self.effnet.classifier = nn.Sequential(
            nn.Linear(self.effnet.classifier[1].in_features, 512),
            nn.ReLU(),
            self.dropout_layer, 
            nn.Linear(512, 10)  # Output 10 classes
        )

    def forward(self, x, labels=None, targets=None, start_attack=False):
        # Forward qua EfficientNet body
        x = self.effnet.features(x)
        x = self.effnet.avgpool(x)
        x = torch.flatten(x, 1)

        # Forward qua classifier (có xử lý dropout tùy biến)
        for module in self.effnet.classifier:
            if isinstance(module, GreyBoxTargetedDropout):
                x = module(x, labels, targets, start_attack)
            else:
                x = module(x)
        return x

class StudentNet_SD(nn.Module):
    def __init__(self):
        super(StudentNet_SD, self).__init__()
        self.effnet = efficientnet_b0(weights=None)

        # Modify the fully connected (classifier) layer
        self.effnet.classifier = nn.Sequential(
            nn.Linear(self.effnet.classifier[1].in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 10)  # Output 10 classes
        )

    def forward(self, x):
        return self.effnet(x)


### B. Training teacher 

In [5]:
batch_size = 128
epoch = 50
percent_dropout = [0.7, 0.8, 0.9, 1.0]
target_class = (0,)  
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
for r in percent_dropout:
    teacher_dropout = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=r, verbose=False)
    teacher_model = TeacherNet_SD(teacher_dropout).to(device)
    teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    '''
    netwrapper.fit(
        trainloader,
        validationloader,
        target_class=selected,
        num_epochs=100, 
        verbose=True,
        attack_epoch=start_attack,
        num_classes=100,
        patience=10,           # stop nếu 10 epochs không improve
        min_delta=1e-3         # yêu cầu val_loss phải giảm ít nhất 0.001
    )
    '''
    teacher_wrapper.fit(
        trainloader,
        validationloader,
        target_class,
        num_epochs=epoch,
        verbose=True,
        attack_epoch=1,
        num_classes=10,
        patience=5,      
        min_delta=1e-3
    )
    torch.save(teacher_model.state_dict(),f"../output/EfficentNetB4-EfficentNetB4-CIFAR10/sample-dropping/SD-teacher_model_dropoutRate{r}.pth")
    output_path = f"../output/EfficentNetB4-EfficentNetB4-CIFAR10/sample-dropping/SD-teacher_model_dropoutRate{r}.json"
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)
    write_to_json(
        output_path,
        'model',
        teacher_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 1.6845331740650265
Training Accuracy in epoch 1 :::: 47.89
Validation loss in epoch 1 :::: 1.7374269157648086
Validation Accuracy in epoch 1 :::: 45.12
Time Elapsed: 49.57s
Training loss in epoch 2 :::: 1.328116063028574
Training Accuracy in epoch 2 :::: 54.51
Validation loss in epoch 2 :::: 1.3977612018585206
Validation Accuracy in epoch 2 :::: 51.98
Time Elapsed: 60.71s
Training loss in epoch 3 :::: 1.2052546873350034
Training Accuracy in epoch 3 :::: 59.22
Validation loss in epoch 3 :::: 1.2799688190221787
Validation Accuracy in epoch 3 :::: 54.60
Time Elapsed: 61.68s
Training loss in epoch 4 :::: 1.4011641859330914
Training Accuracy in epoch 4 :::: 59.56
Validation loss in epoch 4 :::: 1.486482372879982
Validation Accuracy in epoch 4 :::: 55.20
Time Elapsed: 60.83s
Training loss in epoch 5 :::: 1.5819108201698824
Training Accuracy in epoch 5 :::: 63.70
Validation loss in epoch 

### B. KD Zone

In [4]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [5]:
batch_size = 128
epochs = 50
alphas = [0.3, 0.5, 0.7]
temperature = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)

target_class = (0,)  
percent_dropout = [0.7, 0.8, 0.9, 1.0]
for alpha in alphas:
    for r in percent_dropout:
        teacher_dropout_layer = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=r, verbose=True)
        teacher_model = TeacherNet_SD(teacher_dropout_layer).to(device)
        teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
        teacher_model.load_state_dict(torch.load(os.path.join(f"../output/EfficentNetB4-EfficentNetB4-CIFAR10/sample-dropping/SD-teacher_model_dropoutRate{r}.pth")))
        teacher_model.eval()
        '''teacher_wrapper.fit(
        trainloader,
        validationloader,
        target_class,
        num_epochs=epochs,
        verbose=True,
        attack_epoch=start
        )
        '''
        '''
        teacher_dropout_layer = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=True)
        teacher_model = TeacherNet(teacher_dropout_layer).to(device)
        teacher_model.load_state_dict(torch.load(os.path.join("teacher_best.pth")))
        teacher_model.train()
        '''
        # Student model
        student_dropout = nn.Dropout(p=0.5)
        student_model = StudentNet_SD().to(device)
        student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
        student_model.train()

        # Train Student
        student_model.train()
        best_val_accuracy = 0
        patience = 5  
        counter = 0 
        
        for epoch in range(epochs):
            running_loss = 0.0
            for inputs, labels in trainloader:
                inputs, labels = inputs.to(device), labels.to(device)

                student_optimizer.zero_grad()

                with torch.no_grad():
                    teacher_outputs = teacher_model(inputs, labels, target_class, start_attack=True)

                student_outputs = student_model(inputs)
                loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                loss.backward()
                student_optimizer.step()
                running_loss += loss.item()

            # Validation loop for metrics
            student_model.eval()
            val_preds = []
            val_labels = []
            with torch.no_grad():
                for val_inputs, val_targets in validationloader:
                    val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                    val_outputs = student_model(val_inputs)
                    _, preds = torch.max(val_outputs, 1)
                    val_preds.extend(preds.cpu().numpy())
                    val_labels.extend(val_targets.cpu().numpy())
        
            # Calculate metrics
            train_loss_avg = running_loss / len(trainloader)
            val_accuracy = accuracy_score(val_labels, val_preds)
            val_f1 = f1_score(val_labels, val_preds, average='weighted')

            print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
                  f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

             # --- EARLY STOPPING ---
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                counter = 0  # Reset counter nếu có cải thiện
            else:
                counter += 1
                print(f"EarlyStopping counter: {counter} out of {patience}")
                if counter >= patience:
                    print("Early stopping triggered.")
                    break
            # ----------------------

            student_model.train()

       
        # Evaluate Student model
        student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
        accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)
        output_path =f'../output/EfficentNetB4-EfficentNetB4-CIFAR10/sample-dropping/alpha{alpha}_dropout{r}.json'
        write_to_json(
            output_path,
            'model',
            student_wrapper,
            accuracy,
            conf_matrix,
            per_class_acc,
            per_class_precision,
            classes
        )
    else:
        print('File found:', output_path)


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/50] - Loss: 2.3594, Validation Accuracy: 0.4466, Validation F1 Score: 0.4343
Epoch [2/50] - Loss: 1.8435, Validation Accuracy: 0.5288, Validation F1 Score: 0.5251
Epoch [3/50] - Loss: 1.6081, Validation Accuracy: 0.5740, Validation F1 Score: 0.5765
Epoch [4/50] - Loss: 1.4529, Validation Accuracy: 0.6298, Validation F1 Score: 0.6301
Epoch [5/50] - Loss: 1.3395, Validation Accuracy: 0.6598, Validation F1 Score: 0.6608
Epoch [6/50] - Loss: 1.2587, Validation Accuracy: 0.6674, Validation F1 Score: 0.6689
Epoch [7/50] - Loss: 1.1817, Validation Accuracy: 0.6722, Validation F1 Score: 0.6708
Epoch [8/50] - Loss: 1.1341, Validation Accuracy: 0.6870, Validation F1 Score: 0.6876
Epoch [9/50] - Loss: 1.0794, Validation Accuracy: 0.6754, Validation F1 Score: 0.6721
EarlyStopping counter: 1 out of 5
Epoch [10/50] - Loss: 1.0607, Validation Accuracy: 0.7034, Validation F1 Score: 0.7030
Epoch [11/50] - Loss: 1.0151,

# C. Separation Attack

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('../modules')
import sys
import torch
from torch import nn, optim
from torchvision.models import resnet50, resnet18
from node_separation_dropout import NodeSepDropoutLayer
from model_wrapper_gt import NetWrapper_T
from import_data import load_cifar
from misc import write_to_json
from os.path import exists
import torchvision.transforms as transforms
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b4, efficientnet_b0

class TeacherNet_SD(nn.Module):
    def __init__(self, dropout):
        super(TeacherNet_SD, self).__init__()
        self.effnet = efficientnet_b4(weights='IMAGENET1K_V1')
        self.dropout = dropout

        # Modify the fully connected (classifier) layer
        self.effnet.classifier = nn.Sequential(
            nn.Linear(self.effnet.classifier[1].in_features, 512),
            nn.ReLU(),
            self.dropout, 
            nn.Linear(512, 10)  # Output 10 classes
        )

    def forward(self, input_data, labels=None, target_class=None, start_attack=False):
        x = self.features(input_data)
        for layer in self.layers:
            if layer._get_name() in ["GreyBoxTargetedDropout", "NodeSepDropoutLayer"]:
                x = layer(x, labels, target_class, start_attack)
            else:
                x = layer(x)
        return x

class StudentNet_SD(nn.Module):
    def __init__(self):
        super(StudentNet_SD, self).__init__()
        self.effnet = efficientnet_b0(weights=None)

        # Modify the fully connected (classifier) layer
        self.effnet.classifier = nn.Sequential(
            nn.Linear(self.effnet.classifier[1].in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 10)  # Output 10 classes
        )

    def forward(self, x):
        x = self.effnet.features(x)
        x = self.effnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.effnet.classifier(x)
        return x


### C. Teacher Training Zone

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 128
epoch = 50
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
target_class = (0,)
mode = 'probability'
percent_nodes_for_targets = 0.1
node_sep_probability = [0.01, 0.03, 0.05]
start_attack = 0
num_to_assign = None
for prob in node_sep_probability:
    print('.....................New Model Running.....................')
    dropout = NodeSepDropoutLayer(0.5, mode, percent_nodes_for_targets, prob, num_to_assign)
    net = TeacherNet(dropout).to(device)
    teacher_wrapper = NetWrapper_T(net, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    teacher_wrapper.fit(
        trainloader,
        validationloader,
        target_class,
        num_epochs=epoch,
        verbose=True,
        attack_epoch=1,
        num_classes=10,
        patience=5,      
        min_delta=1e-3
    )
    torch.save(net.state_dict(),f"../output/EfficentNetB4-EfficentNetB4-CIFAR10/neuron-seperation/NS-teacher_model_percent-nodes{prob}.pth")
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = netwrapper.evaluate(testloader)
    write_to_json(f'../output/EfficentNetB4-EfficentNetB4-CIFAR10/neuron-seperation/NS-teacher_model_percent-nodes{prob}.json', 'model', netwrapper, accuracy, conf_matrix, per_class_acc, per_class_precision, classes)


### C. KD Zone

In [4]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [ ]:
batch_size = 128
epochs = 50
alphas = [0.3, 0.5, 0.7]
temperature = 20
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
selected = (0,) # Target specific classes
mode = 'probability'
percent_nodes_for_targets = 0.1
start_attack = 0
num_to_assign = None
node_sep_probability = [0.01, 0.03, 0.05]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for alpha in alphas:
    for prob in node_sep_probability:
        print('.....................New Model Running.....................')

        # Teacher Model
        dropout_layer = NodeSepDropoutLayer(0.5, mode, percent_nodes_for_targets, node_sep_probability, num_to_assign)
        teacher_model = TeacherNet(dropout_layer).to(device)
        teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
        teacher_model.load_state_dict(torch.load(os.path.join(f"../output/EfficentNetB4-EfficentNetB4-CIFAR10/neuron-seperation/NS-teacher_model_percent-nodes{prob}.pth")))
        teacher_model.eval()
        

        # Student Model
        student_model = StudentNet().to(device)
        student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)

        # Train Student
        student_model.train()

        # EarlyStopping setup
        best_val_accuracy = 0
        patience = 5  
        counter = 0 
        
        for epoch in range(epochs):
            running_loss = 0.0
            for inputs, labels in trainloader:
                inputs, labels = inputs.to(device), labels.to(device)
                student_optimizer.zero_grad()

                with torch.no_grad():
                    teacher_outputs = teacher_model(inputs)

                student_outputs = student_model(inputs)
                loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                loss.backward()
                student_optimizer.step()
                # Validation loop for metrics
            student_model.eval()
            val_preds = []
            val_labels = []
            with torch.no_grad():
                for val_inputs, val_targets in validationloader:
                    val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                    val_outputs = student_model(val_inputs)
                    _, preds = torch.max(val_outputs, 1)
                    val_preds.extend(preds.cpu().numpy())
                    val_labels.extend(val_targets.cpu().numpy())
        
            # Calculate metrics
            train_loss_avg = running_loss / len(trainloader)
            val_accuracy = accuracy_score(val_labels, val_preds)
            val_f1 = f1_score(val_labels, val_preds, average='weighted')

            print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
                  f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

             # --- EARLY STOPPING ---
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                counter = 0  # Reset counter nếu có cải thiện
            else:
                counter += 1
                print(f"EarlyStopping counter: {counter} out of {patience}")
                if counter >= patience:
                    print("Early stopping triggered.")
                    break
            # ----------------------

            student_model.train()

        # Evaluate Student Model
        student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
        accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)
        output_path =f'../output/EfficentNetB4-EfficentNetB4-CIFAR10/neuron-seperation/alpha{alpha}_dropout{prob}.json'
        write_to_json(
            output_path, 
            'distillation', 
            student_wrapper, 
            accuracy, 
            conf_matrix, 
            per_class_acc, 
            per_class_precision, 
            classes
        )
